In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import  matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error

from random import sample
seed_list = list(range(10000))

import warnings
warnings.simplefilter('ignore')
os.chdir('C:\\Users\\yeonjun.in\\Desktop\\연준\\캐글\\제주도\\data')

TODAY = str(datetime.now().year)+str(datetime.now().month)+str(datetime.now().day)

In [2]:
os.chdir('C:\\Users\\yeonjun.in\\Desktop\\연준\\캐글\\제주도\\data')

train = pd.read_pickle('fixed2_train.pkl')
test = pd.read_pickle("fixed2_test.pkl")
input_var = pd.read_csv('fixed2_input_var.csv')

sub = pd.read_csv('submission_sample.csv')

os.chdir('C:\\Users\\yeonjun.in\\Desktop\\연준\\캐글\\제주도\\data\\raw')
bus = pd.read_csv('bus_bts.csv')

os.chdir('C:\\Users\\yeonjun.in\\Desktop\\연준\\캐글\\제주도\\code\\experiment')
experiment_db = pd.read_csv('experiment_DB.csv')

In [3]:
added = []
input_var = list(input_var.columns)
target=['ride_18_20']

In [4]:
all = pd.concat([train,test],axis=0).reset_index(drop=True)

all['in_out_bus_route_id'] = all['bus_route_id'].astype('str') + all['in_out'].astype('str')

temp = all.groupby('in_out_bus_route_id')['id'].count().to_dict()

all['inout_bus_route_id_freq'] = all['in_out_bus_route_id'].map(temp)

temp = all.groupby('in_out_bus_route_id')['ride_6_12'].agg(['mean','min','max','sum']).rename(
columns = {
    'mean' : 'inout_bus_route_id'+ '_' + 'ride_6_12' +'_'+'mean',
    'min' : 'inout_bus_route_id'+ '_' + 'ride_6_12' +'_'+'min',
    'max' : 'inout_bus_route_id'+ '_' + 'ride_6_12' +'_'+'max',
    'sum' : 'inout_bus_route_id'+ '_' + 'ride_6_12' +'_'+'sum'
})

all = pd.merge(all,temp,how='left',on='in_out_bus_route_id')

temp = all.groupby('in_out_bus_route_id')['takeoff_6_12'].agg(['mean','min','max','sum']).rename(
columns = {
    'mean' : 'inout_bus_route_id'+ '_' + 'takeoff_6_12' +'_'+'mean',
    'min' : 'inout_bus_route_id'+ '_' + 'takeoff_6_12' +'_'+'min',
    'max' : 'inout_bus_route_id'+ '_' + 'takeoff_6_12' +'_'+'max',
    'sum' : 'inout_bus_route_id'+ '_' + 'takeoff_6_12' +'_'+'sum'
})

all = pd.merge(all,temp,how='left',on='in_out_bus_route_id')

train = all.loc[:(train.shape[0]-1),]
test = all.loc[train.shape[0]:,].drop('ride_18_20',axis=1).reset_index(drop=True)

del all

added += [a for a in train.columns if 'inout_' in a]

In [5]:
all = pd.concat([train,test],axis=0).reset_index(drop=True)
all['diff_ride_takeoff'] = all['ride_6_12'] - all['takeoff_6_12']

train = all.loc[:(train.shape[0]-1),]
test = all.loc[train.shape[0]:,].drop('ride_18_20',axis=1).reset_index(drop=True)

del all

added += [a for a in train.columns if 'diff_ride_takeoff' in a]

In [6]:
all = pd.concat([train,test],axis=0).reset_index(drop=True)

all['bus_route_id_station_code_concat'] = str(all['station_code']) + str(all['bus_route_id'])

train = all.loc[:(train.shape[0]-1),]
test = all.loc[train.shape[0]:,].drop('ride_18_20',axis=1).reset_index(drop=True)

del all

In [7]:
# all = pd.concat([train,test],axis=0).reset_index(drop=True)

# all['weekday_station_code_concat'] = str(all['station_code']) + str(all['weekday_var'])

# train = all.loc[:(train.shape[0]-1),]
# test = all.loc[train.shape[0]:,].drop('ride_18_20',axis=1).reset_index(drop=True)

# del all

In [8]:
all = pd.concat([train,test],axis=0).reset_index(drop=True)

temp = all.groupby('station_name')['station_code'].nunique().reset_index().rename(columns={'station_code':'code_num_per_name'})

all = pd.merge(all,temp,how='left',on='station_name')

train = all.loc[:(train.shape[0]-1),]
test = all.loc[train.shape[0]:,].drop('ride_18_20',axis=1).reset_index(drop=True)

del all

# added += [a for a in train.columns if 'code_num_per_name' in a]

In [9]:
# all = pd.concat([train,test],axis=0).reset_index(drop=True)

# all['station_travel'] = np.where(all['station_name'].str.contains('호텔'),'hotel',
#         np.where(all['station_name'].str.contains('콘도'),'condo',
#         np.where(all['station_name'].str.contains('공항'),'airport',
#         np.where(all['station_name'].str.contains('터미널'),'terminal',
#         np.where(all['station_name'].str.contains('초등학교'),'cho',
#                 np.where(all['station_name'].str.contains('중학교'),'jung',
#                 np.where(all['station_name'].str.contains('고등학교'),'ko',
#                         np.where(all['station_name'].str.contains('대학'),'dae',
#                                  np.where(all['station_name'].str.contains('환승'),'hwan',
#                                           np.where(all['station_name'].str.contains('거리'),'street','etc'))))))))))


# all = pd.concat([all,temp],axis=1)

# train = all.loc[:(train.shape[0]-1),]
# test = all.loc[train.shape[0]:,].drop('ride_18_20',axis=1).reset_index(drop=True)

# del all

In [10]:
# all = pd.concat([train,test],axis=0).reset_index(drop=True)

# all['6~7_diff_rt'] = all['ride_6_7'] - all['takeoff_6_7']
# all['7~8_diff_rt'] = all['ride_7_8'] - all['takeoff_7_8']
# all['8~9_diff_rt'] = all['ride_8_9'] - all['takeoff_8_9']
# all['9~10_diff_rt'] = all['ride_9_10'] - all['takeoff_9_10']
# all['10~11_diff_rt'] = all['ride_10_11'] - all['takeoff_10_11']
# all['11~12_diff_rt'] = all['ride_11_12'] - all['takeoff_11_12']

# train = all.loc[:(train.shape[0]-1),]
# test = all.loc[train.shape[0]:,].drop('ride_18_20',axis=1).reset_index(drop=True)

# del all

# added += [a for a in train.columns if 'diff_rt' in a]

In [11]:
# all = pd.concat([train,test],axis=0).reset_index(drop=True)

# all['6~7_plus_rt'] = all['ride_6_7'] + all['takeoff_6_7']
# all['7~8_plus_rt'] = all['ride_7_8'] + all['takeoff_7_8']
# all['8~9_plus_rt'] = all['ride_8_9'] + all['takeoff_8_9']
# all['9~10_plus_rt'] = all['ride_9_10'] + all['takeoff_9_10']
# all['10~11_plus_rt'] = all['ride_10_11'] + all['takeoff_10_11']
# all['11~12_plus_rt'] = all['ride_11_12'] + all['takeoff_11_12']

# train = all.loc[:(train.shape[0]-1),]
# test = all.loc[train.shape[0]:,].drop('ride_18_20',axis=1).reset_index(drop=True)

# del all

# added += [a for a in train.columns if 'plus_rt' in a]

In [12]:
# all = pd.concat([train,test],axis=0).reset_index(drop=True)

# all['all_6_12'] =  all['ride_6_12'] + all['takeoff_6_12']

# all['all_6_12_ride*takeoff'] = all['ride_6_12']*all['takeoff_6_12']
# # all['all_6_12_ride*/takeoff'] = (all['ride_6_12'])/(all['takeoff_6_12']+1)

# train = all.loc[:(train.shape[0]-1),]
# test = all.loc[train.shape[0]:,].drop('ride_18_20',axis=1).reset_index(drop=True)

# del all

# added += [a for a in train.columns if 'all_6_12' in a]

In [13]:
# all = pd.concat([train,test],axis=0).reset_index(drop=True)

# temp = bus.groupby('bus_route_id')['vhc_id'].nunique().to_dict()
# all['vhc_id_num'] = all['bus_route_id'].map(temp).fillna(0)

# temp = all.groupby('vhc_id_num')['id'].count().to_dict()

# all['vhc_id_num_freq'] = all['vhc_id_num'].map(temp).fillna(0)

# train = all.loc[:(train.shape[0]-1),]
# test = all.loc[train.shape[0]:,].drop('ride_18_20',axis=1).reset_index(drop=True)

# del all

# added += [a for a in train.columns if 'vhc_id_num' in a]

In [14]:
# from haversine import haversine
# all = pd.concat([train,test],axis=0).reset_index(drop=True)

# dis = all[['station_code','latitude','longitude']].drop_duplicates().reset_index(drop=True)

# start = datetime.now()
# near = []
# for aa,bb in zip(dis.latitude, dis.longitude):    
#     dis_ar = np.array([haversine((aa,bb),(a,b)) for a,b in zip(dis.latitude, dis.longitude)])
#     temp = np.where(dis_ar == 0,100.0,dis_ar)
#     near += [dis.loc[np.argmin(temp),'station_code']]
    
# end = datetime.now()
# print(end - start)

# dis['near_station_code'] = near

# temp = pd.merge(dis,
#          train.groupby('station_code')['takeoff_6_12'].mean().reset_index().\
#          rename(columns={'station_code' : 'near_station_code'}),how='left',on='near_station_code')[['station_code','takeoff_6_12']].\
# rename(columns= {'takeoff_6_12':'takeoff_6_12_y'})

# all = pd.merge(all,temp,how='left',on='station_code')

# all['takeoff_6_12_y'] = all['takeoff_6_12_y'].fillna(0)

# train = all.loc[:(train.shape[0]-1),]
# test = all.loc[train.shape[0]:,].drop('ride_18_20',axis=1).reset_index(drop=True)

# del all

# added += [a for a in train.columns if 'takeoff_6_12_y' in a]

In [15]:
all = pd.concat([train,test],axis=0).reset_index(drop=True)

code_which = all[['station_code','latitude','longitude']].drop_duplicates().reset_index(drop=True).rename(columns = {
    'station_code' : 'geton_station_code',
    'latitude' : 'geton_lat',
    'longitude' : 'geton_long'
})
bus = pd.merge(bus,code_which, how='left',on='geton_station_code')

code_which = all[['station_code','latitude','longitude']].drop_duplicates().reset_index(drop=True).rename(columns = {
    'station_code' : 'getoff_station_code',
    'latitude' : 'getoff_lat',
    'longitude' : 'getoff_long'
})
bus = pd.merge(bus,code_which, how='left',on='getoff_station_code')

all = pd.merge(all,pd.DataFrame({'station_code':bus[bus['geton_station_code'] == bus['getoff_station_code']].geton_station_code.unique(),
             'same_on_off' : 1}),how='left',on='station_code') 

all['same_on_off'] = all['same_on_off'].fillna(0)


In [16]:
geton = []
getoff = []
for aa,bb,cc,dd in zip(bus['geton_lat'],bus['geton_long'],bus['getoff_lat'],bus['getoff_long']):
    a = (aa,bb)
    b = (cc,dd)
    geton += [a]
    getoff += [b]
    
from haversine import haversine

dis = []
for on,off in zip(geton,getoff):
    dis += [haversine(on,off)]
    
bus['moving_dis'] = dis

temp = bus.groupby('bus_route_id')['moving_dis'].mean().fillna(0).to_dict()

all['moving_dis_per_bus'] = all['bus_route_id'].map(temp)

temp = bus.groupby('geton_station_code')['moving_dis'].mean().fillna(0).to_dict()

all['moving_dis_per_geton'] = all['station_code'].map(temp)

temp = bus.groupby('getoff_station_code')['moving_dis'].mean().fillna(0).to_dict()

all['moving_dis_per_getoff'] = all['station_code'].map(temp)

all['moving_dis_per_bus'] =  all['moving_dis_per_bus'].fillna(all['moving_dis_per_bus'].median()) 
all['moving_dis_per_getoff'] = all['moving_dis_per_getoff'].fillna(all['moving_dis_per_getoff'].median()) 
all['moving_dis_per_geton'] = all['moving_dis_per_geton'].fillna(all['moving_dis_per_geton'].median()) 

train = all.loc[:(train.shape[0]-1),]
test = all.loc[train.shape[0]:,].drop('ride_18_20',axis=1).reset_index(drop=True)

del all

added += [a for a in train.columns if 'moving_dis' in a]

In [17]:
all = pd.concat([train,test],axis=0).reset_index(drop=True)

bus['travel'] = bus['user_card_id'].map(bus['user_card_id'].value_counts()[bus['user_card_id'].value_counts()<10].to_dict())
bus['travel'] = np.where(bus['travel'].isnull(),1,0)
temp = bus.groupby('geton_station_code')['travel'].sum().to_dict()

all['travel1'] = all['station_code'].map(temp).fillna(0)


bus['travel'] = bus['user_card_id'].map(bus['user_card_id'].value_counts()[bus['user_card_id'].value_counts()<10].to_dict())
bus['travel'] = np.where(bus['travel'].isnull(),1,0)
temp = bus.groupby('getoff_station_code')['travel'].sum().to_dict()

all['travel2'] = all['station_code'].map(temp).fillna(0)

bus['travel'] = bus['user_card_id'].map(bus['user_card_id'].value_counts()[bus['user_card_id'].value_counts()<10].to_dict())
bus['travel'] = np.where(bus['travel'].isnull(),0,1)
temp = bus.groupby('geton_station_code')['travel'].sum().to_dict()

all['travel3'] = all['station_code'].map(temp).fillna(0)

bus['travel'] = bus['user_card_id'].map(bus['user_card_id'].value_counts()[bus['user_card_id'].value_counts()<10].to_dict())
bus['travel'] = np.where(bus['travel'].isnull(),0,1)
temp = bus.groupby('getoff_station_code')['travel'].sum().to_dict()

all['travel4'] = all['station_code'].map(temp).fillna(0)

# all['travel1'] = round(all['travel1']/(all['travel1'].max()),4)
# all['travel2'] = round(all['travel2']/(all['travel2'].max()),4)
# all['travel3'] = round(all['travel3']/(all['travel3'].max()),4)
# all['travel4'] = round(all['travel4']/(all['travel4'].max()),4)



train = all.loc[:(train.shape[0]-1),]
test = all.loc[train.shape[0]:,].drop('ride_18_20',axis=1).reset_index(drop=True)

del all, bus

added += ['travel1','travel2','travel3','travel4']

In [18]:
all = pd.concat([train,test],axis=0).reset_index(drop=True)

temp = np.power(all[['ride_6_7','ride_7_8','ride_8_9','ride_9_10','ride_10_11','ride_11_12','takeoff_6_7','takeoff_7_8','takeoff_8_9','takeoff_9_10','takeoff_10_11','takeoff_11_12']],2)
temp.columns = [a+'_power' for a in tuple(temp.columns)]

all = pd.concat([all,temp],axis=1)

train = all.loc[:(train.shape[0]-1),]
test = all.loc[train.shape[0]:,].drop('ride_18_20',axis=1).reset_index(drop=True)

del all

added += [a for a in train.columns if 'power' in a]

In [19]:
import geopy.distance 

all = pd.concat([train,test],axis=0).reset_index(drop=True)

jeju=(33.51411, 126.52969) # 제주 측정소 근처
gosan=(33.29382, 126.16283) #고산 측정소 근처
seongsan=(33.38677, 126.8802) #성산 측정소 근처
po=(33.24616, 126.5653) #서귀포 측정소 근처

t1 = [geopy.distance.vincenty( (i,j), jeju).km for i,j in list( zip( all['latitude'],all['longitude'] )) ]
t2 = [geopy.distance.vincenty( (i,j), gosan).km for i,j in list( zip( all['latitude'],all['longitude'] )) ]
t3 = [geopy.distance.vincenty( (i,j), seongsan).km for i,j in list( zip( all['latitude'],all['longitude'] )) ]
t4 = [geopy.distance.vincenty( (i,j), po).km for i,j in list( zip( all['latitude'],all['longitude'] )) ]

all['dis_jeju']=t1
all['dis_gosan']=t2
all['dis_seongsan']=t3
all['dis_po']=t4

all['dist_name'] = all[['dis_jeju','dis_gosan','dis_seongsan','dis_po']].apply(lambda x: np.argmin(x),axis=1).str.slice(4,)

In [20]:
added += ['dis_jeju','dis_gosan','dis_seongsan','dis_po']

In [21]:
input_var = input_var + added

In [22]:
os.getcwd()

'C:\\Users\\yeonjun.in\\Desktop\\연준\\캐글\\제주도\\code\\experiment'

In [23]:
os.chdir('C:\\Users\\yeonjun.in\\Desktop\\연준\\캐글\\제주도\\data')
all.loc[:(train.shape[0]-1),].to_pickle('fixed3_train.pkl')
all.loc[train.shape[0]:,].drop('ride_18_20',axis=1).reset_index(drop=True).to_pickle('fixed3_test.pkl')

pd.DataFrame(columns = input_var).to_csv('fixed3_input_var.csv',index=False)